In [ ]:
# using Pkg
# Pkg.activate(".")
# Pkg.instantiate()
# Pkg.add("SQLite")
# Pkg.add("DBInterface")
# Pkg.add("JSON3")
# Pkg.add("EasyConfig")
# Pkg.add("MurmurHash3")
# Pkg.add("TextAnalysis")
# Pkg.add("UUIDs")
# Pkg.add("PooledArrays")
include("src/lisa_store.jl")

using SQLite
using DBInterface
using MurmurHash3
using TextAnalysis
using JSON3
using PooledArrays
using UUIDs
using EasyConfig
using HDF5

db = Graph.DB("lisa_analytics.db")
hll = SetCore.HllSet{10}()

# Create an empty HDF5 file. Run it only once, because it will overwrite the file
# h5open("lisa_analytics.hdf5", "w") do f
#     # The file is now open, but it's empty
# end

hdf5_file_name = "lisa_analytics.hdf5"

In [ ]:
p = 10
hll = SetCore.HllSet{p}()
Store.book_file(db, "/home/alexmy/JULIA/DEMO/sample/", hll)

In [ ]:
uuid = string(uuid4())
df = Graph.set_lock!(db, 
    "/home/alexmy/JULIA/DEMO/sample", 
    "csv", 
    "book_file", 
    "ingest_csv", 
    "waiting", 
    "waiting", 
    uuid; result=true)

for row in eachrow(df)
    assign = Graph.Assignment(row) 
    col_uuid = string(uuid4())
    Store._ingest_csv_by_column(db, assign, col_uuid; limit=10000, offset=20)
end

In [ ]:
Store.commit(db, hdf5_file_name, "Alex Mylnikov", "alexmy@lisa-park.com", "commit 3", Config())

In [ ]:
analytics = h5open(hdf5_file_name)

In [ ]:
close(analytics)

In [ ]:
dataset_path = "/afbc73c3-62d1-4e70-aa89-b5affe924104/nodes/0f73a283cd731d2f36ef6b029f24d28458b8fefc/_csv_column_"
data_out = Dict()
h5open(hdf5_file_name, "r") do file
    Store.read_datasets(file, data_out, dataset_path)
end
println(data_out["_csv_column_"][1])